In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
import joblib

In [2]:
tr = pd.read_csv(r"C:\Users\lenovo\Documents\Uma\data\bigmart sales hackathon data\train_v9rqX0R.csv")
te = pd.read_csv(r"C:\Users\lenovo\Documents\Uma\data\bigmart sales hackathon data\test_AbJTz2l.csv")
submission = pd.read_csv(r"C:\Users\lenovo\Documents\Uma\data\bigmart sales hackathon data\sample_submission_8RXa3c6.csv")

In [3]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [4]:
te.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5681 entries, 0 to 5680
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            5681 non-null   object 
 1   Item_Weight                4705 non-null   float64
 2   Item_Fat_Content           5681 non-null   object 
 3   Item_Visibility            5681 non-null   float64
 4   Item_Type                  5681 non-null   object 
 5   Item_MRP                   5681 non-null   float64
 6   Outlet_Identifier          5681 non-null   object 
 7   Outlet_Establishment_Year  5681 non-null   int64  
 8   Outlet_Size                4075 non-null   object 
 9   Outlet_Location_Type       5681 non-null   object 
 10  Outlet_Type                5681 non-null   object 
dtypes: float64(3), int64(1), object(7)
memory usage: 488.3+ KB


In [5]:
tr.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [6]:
te.duplicated().sum()

0

In [7]:
tr['Item_Weight'].fillna(tr['Item_Weight'].mean(), inplace = True)

In [8]:
tr['Outlet_Size'].fillna('Unknown', inplace = True)

In [9]:
te['Item_Weight'].fillna(te['Item_Weight'].mean(), inplace = True)

In [10]:
te['Outlet_Size'].fillna('Unknown', inplace = True)

In [11]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                8523 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [12]:
tr['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [13]:
tr['Item_Fat_Content'].replace({'LF': 'Low Fat', 'low fat': 'Low Fat', 'reg': 'Regular'}, inplace = True)
te['Item_Fat_Content'].replace({'LF': 'Low Fat', 'low fat': 'Low Fat', 'reg': 'Regular'}, inplace = True)

In [14]:
num = tr.select_dtypes(include = np.number).drop(columns = ['Item_Outlet_Sales'])
num[0:2]

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year
0,9.30,0.016047,249.8092,1999
1,5.92,0.019278,48.2692,2009


In [15]:
cat = tr.select_dtypes(exclude = np.number)
cat[0:2]

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDA15,Low Fat,Dairy,OUT049,Medium,Tier 1,Supermarket Type1
1,DRC01,Regular,Soft Drinks,OUT018,Medium,Tier 3,Supermarket Type2


In [16]:
std_scale = StandardScaler()
numerical = std_scale.fit_transform(num)
numerical_tr = pd.DataFrame(numerical, columns = num.columns)

In [17]:
cat['Item_Type'] = cat['Item_Type'].astype('category').cat.codes

<ipython-input-17-1ab76cb0e230>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat['Item_Type'] = cat['Item_Type'].astype('category').cat.codes


In [18]:
cat['Outlet_Size'] = cat['Outlet_Size'].astype('category').cat.codes

<ipython-input-18-f77353d2f710>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat['Outlet_Size'] = cat['Outlet_Size'].astype('category').cat.codes


In [19]:
cat['Outlet_Location_Type'] = cat['Outlet_Location_Type'].astype('category').cat.codes

<ipython-input-19-71bc1058814e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat['Outlet_Location_Type'] = cat['Outlet_Location_Type'].astype('category').cat.codes


In [20]:
cat['Outlet_Type'] = cat['Outlet_Type'].astype('category').cat.codes

<ipython-input-20-02b016f03c7f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat['Outlet_Type'] = cat['Outlet_Type'].astype('category').cat.codes


In [21]:
cat['Item_Fat_Content'].replace({'Low Fat': 0}, inplace = True)

C:\Users\lenovo\anaconda3\lib\site-packages\pandas\core\series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [22]:
cat['Item_Fat_Content'].replace({'Regular': 1}, inplace = True)

In [23]:
cat[0:2]

,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDA15,0,4,OUT049,1,0,1
1,DRC01,1,14,OUT018,1,2,2


In [24]:
tr_pre = pd.concat([numerical_tr, cat, tr[['Item_Outlet_Sales']]], axis = 1)
tr_pre.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,-0.841872,-0.970732,1.747454,0.139541,FDA15,0,4,OUT049,1,0,1,3735.1380
1,-1.641706,-0.908111,-1.489023,1.334103,DRC01,1,14,OUT018,1,2,2,443.4228
2,1.098554,-0.956917,0.010040,0.139541,FDN15,0,10,OUT049,1,0,1,2097.2700
3,1.500838,-1.281758,0.660050,0.020085,FDX07,1,6,OUT010,3,2,0,732.3800
4,-0.929428,-1.281758,-1.399220,-1.293934,NCD19,0,9,OUT013,0,2,1,994.7052


In [25]:
x_train = tr_pre.drop(columns = ['Item_Identifier', 'Outlet_Identifier', 'Item_Outlet_Sales'])
y_train = tr_pre['Item_Outlet_Sales']
model1 = LinearRegression()
model1.fit(x_train, y_train)
y_tr_pred = model1.predict(x_train)
print(mean_squared_error(y_train, y_tr_pred, squared = False))

1206.6814466961985


In [26]:
num1 = te.select_dtypes(include = np.number)
cat1 = te.select_dtypes(exclude = np.number)
std_scale1 = StandardScaler()
numerical1 = std_scale1.fit_transform(num1)
numerical_te = pd.DataFrame(numerical1, columns = num1.columns)

In [27]:
cat1['Item_Type'] = cat1['Item_Type'].astype('category').cat.codes
cat1['Outlet_Size'] = cat1['Outlet_Size'].astype('category').cat.codes
cat1['Outlet_Location_Type'] = cat1['Outlet_Location_Type'].astype('category').cat.codes
cat1['Outlet_Type'] = cat1['Outlet_Type'].astype('category').cat.codes
cat1['Item_Fat_Content'].replace({'Low Fat': 0}, inplace = True)
cat1['Item_Fat_Content'].replace({'Regular': 1}, inplace = True)

<ipython-input-27-91fd917c4067>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat1['Item_Type'] = cat1['Item_Type'].astype('category').cat.codes
<ipython-input-27-91fd917c4067>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat1['Outlet_Size'] = cat1['Outlet_Size'].astype('category').cat.codes
<ipython-input-27-91fd917c4067>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [28]:
te_pre = pd.concat([numerical_te, cat1], axis = 1)
te_pre.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Identifier,Item_Fat_Content,Item_Type,Outlet_Identifier,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,1.897460e+00,-1.134077,-0.536555,0.139891,FDW58,0,13,OUT049,1,0,1
1,-1.035530e+00,-0.531850,-0.868937,1.095512,FDW14,1,4,OUT017,3,1,1
2,4.486336e-01,0.661316,1.629848,0.020438,NCN55,0,11,OUT010,3,2,0
3,-1.267578e+00,-0.981416,0.226697,1.095512,FDQ58,0,13,OUT017,3,1,1
4,-5.230960e-14,1.032540,1.508110,-1.532446,FDY38,1,4,OUT027,1,2,3


In [29]:
x_test = te_pre.drop(columns = ['Item_Identifier', 'Outlet_Identifier'])
y_te_pred = model1.predict(x_test)

In [30]:
submission['Item_Identifier'] = te_pre['Item_Identifier']
submission['Outlet_Identifier'] = te_pre['Outlet_Identifier']
submission['Item_Outlet_Sales'] = np.abs(y_te_pred)

In [31]:
submission.to_csv('bigmart benchmark.csv', index = False)

In [32]:
joblib.dump(model1, 'bigmart.pkl')

['bigmart.pkl']